# MongoDB

---

**Источники:**

[MongoDB](https://ru.wikipedia.org/wiki/MongoDB)

[Большой туториал по MongoDB](https://medium.com/@Merrick_krg/%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%BE%D0%B9-%D1%82%D1%83%D1%82%D0%BE%D1%80%D0%B8%D0%B0%D0%BB-%D0%BF%D0%BE-mongodb-7e2f9e17f0c7)

[MongoDB: Создание, обновление и удаление документов](https://habr.com/ru/post/134524/)

[MongoDB: Запросы](https://habr.com/ru/post/134590/)

[MongoDB vs MySQL: A Comparative Study on Databases](https://www.simform.com/mongodb-vs-mysql-databases)

[Academy 3T](https://studio3t.com/academy/)

[Robo 3T GUI](https://robomongo.org/download)

[PyMongo 3.11.2 documentation Tutorial](https://pymongo.readthedocs.io/en/stable/tutorial.html)

---

## Терминология

**MongoDB** — документоориентированная система управления базами данных, не требующая описания схемы таблиц.

Считается одним из классических примеров NoSQL-систем. 

Использует JSON-подобные документы и схему базы данных.

Написана на языке C++. 

Система поддерживает ad-hoc-запросы: они могут возвращать конкретные поля документов и пользовательские JavaScript-функции. 

*Ad hoc — латинская фраза, означающая «специально для этого», «по особому случаю».

Поддерживается поиск по регулярным выражениям. Также можно настроить запрос на возвращение случайного набора результатов.

Имеется поддержка индексов (объект базы данных, создаваемый с целью повышения производительности поиска данных).

<center><img src="images/MongoDB_database_colection.png"/></center>\

В июне 2018 года (в версии 4.0) добавлена поддержка транзакций, удовлетворяющих требованиям ACID.

### MongoDB подходит для следующих применений
- хранение и регистрация событий;
- системы управления документами и контентом;
- электронная коммерция;
- игры;
- данные мониторинга, датчиков;
- мобильные приложения;
- хранилище операционных данных веб-страниц (например, хранение комментариев, рейтингов, профилей пользователей, сеансы пользователей).

### Различия MongoDB и RDBMS

<center><img src="images/rdbms_vs_mongodb.png"/></center>

<center><img src="images/mongodb_and_rdbms_schema.png"/></center>

---

## Подготовка окружения

### Установка MongoDB

[MongoDB Community Server](https://www.mongodb.com/try/download/community)

[Robo 3T: the hobbyist GUI](https://robomongo.org/download)

In [ ]:
!which pip

In [ ]:
!conda install pymongo -y

---

## MongoDB Типы данных

Документы в MongoDB являются **"JSON-like"**.

- `null`. Может использоваться для представления нулевого значения или несуществующего поля.
```
{ x: null }
```


- `boolean`. Это два значения true false.
```
{ x: true, y: false }
```


- `number`. Числа используются формат 64 битных с плавающей точкой. Для целых чисел можно использовать функции `NumberInt()` или `NumberLong()` которые используют 4 байта и 8 байтов для хранения чисел соответственно. `NumberDecimal()` принимает десятичное значение в виде строки.
```
{
 x: 3.14,
 y: 3,
 z: NumberInt('3'),
 r: NumberLong('2090845886852'),
 m: NumberDecimal("1000.55")
}
```


- `string`. Строки это массив UTF-8 символов.
```
{ x: 'string' }
```


- `date`. Дата в mongoDB это 64-битные целые числа, которые показывают дату прошедшую с эпохи Unix (англ. Unix Epoch) (1 января, 1970 года). Time zone не сохраняется. Для работы с датой mongoDB используют класс Date в JS. Дата в mongo shell используют текущую time zone, но в базе хранится значение миллисекунд прошедших с эпохи линукс.
```
{ 
x: new Date(),
y: new ISODate("2012-12-19T06:01:17.171Z")
}
```


- `regular expression`. Сохраненные регулярные выражения могут использоваться в регулярных выражениях в JS.
```
{ x: /foobar/i }
```


- `array`. Наборы (sets) или Списки (lists) могут быть представлены как массивы в базе. MongoDB поддерживает атомный update элементов. Например если в целом массиве нужно заменить элемент pie на pi, то мы заменим только этот элемент.
```
{ x: ['string', 3.14, new Date()] }
```


- `embedded document`. Документы могут содержать целые документы, встроенные в качестве значений в родительский документ. MongoDB так же как и с массивами понимает, когда используется embedded documents и оптимизирует работу с ними.
```
{ x: { name: 'Merrick', isAdmin: true } }
```


- `object id`. Идентификатор объекта это отдельный 12-байтовый тип данных, который используется для идентифицирования документов.
    - Каждый документ должен иметь поле _id, поле может быть любого типа, но обычно оно типа `ObjectId`. 
    - Тип `ObjectId` был разработан, легким (lightweight) в тоже время, что бы мог генерировать уникальные ключи на разных платформах. 
    - Если создать несколько `object id` в быстрой последовательности, можно заметить, что только последние несколько цифр меняются каждый раз. Это связано с тем как создаются `ObjectId`. 12 байтов формируются следующим образом:
        - Первые 4 байта это timestamp. В этой информации присутствует неявно дата создания документа.
        - Следующие 3 байта это machine PID, что означает уникальный идентификатор машины, сделано для того, чтобы разные машины случайно не создали одинаковый `ObjectId`. Обычно это хэш машины.
        - Чтобы обеспечить уникальность между процессами, следующие два байта берутся из PID процесса.
        - Остальные три байта это просто инкрементация, которая отвечает за уникальность выполнения в данную секунду, что позволяет нам создавать 16,777,216 `ObjectId` в секунду на одной машине в одном процессе.

```
{ x: ObjectId() }
{ _id: 10, calc: NumberLong("2090845886852") }
```


- `binary data`. Двоичные данные представляют собой строку произвольных байтов. Ими нельзя манипулировать из оболочки. Двоичные данные - единственный способ хранить строки не-UTF-8 в базе данных.


- `code`. MongoDB позволяет хранить произвольный JS код в документах и запросах под отдельным типом данных..
```
{ x: function list() { /* ... */ } }
```

---

## MongoDB + Python

In [ ]:
import pymongo
from pymongo import MongoClient

pymongo.__version__

---

### Подключение к базе данных

In [ ]:
# данные для подключения к БД
host='localhost'
port=3306

In [ ]:
# альтернативный вариант: MongoClient(host, port)
client = MongoClient(f'mongodb://{host}:{port}/') 
client

In [ ]:
db = client['test-database']
db

In [ ]:
# альтернативный вариант: collection = db['test_collection']
collection = db.test_collection
collection

---

### Создание, обновление и удаление документов. Запросы.

Данные в MongoDB представлены (и хранятся) с использованием документов в стиле JSON.

В PyMongo используются словари (dict) для представления документов.

- `insert` - базовый метод для добавление информации.
- `remove` - базовый метод для добавление информации.
- `update` - базовый метод для обновления информации.
- `find` - аналог SELECT в MySQL, используется для выборки документов.

Продолжение в файле [6_MongoDB_client.py](./6_MongoDB_client.py)